In [ ]:
from datetime import datetime

import metpy.calc as mpcalc
import metpy.constants as mpconst
from metpy.units import units
import numpy as np
import pint
import xarray as xr

In [ ]:
def read_gdlist(name):
    file = open(name, 'rb')

    for _ in range(6):
        file.readline()

    grid_row = file.readline()
    nlon = int(grid_row[-11:-5].strip())
    nlat = int(grid_row[-4:].strip())
    #print(nlon,nlat)

    gempak_array = np.empty((nlat,nlon))

    for _ in range(2):
        file.readline()

    scale_row = file.readline()
    scale_factor = int(scale_row[19:].strip())
    #print(scale_factor)

    for _ in range(92):
        file.readline()

    col = 0
    row = 0
    for line in file.readlines():
        if (line[:13].strip() != b'') & (col >= nlon):
            col = 0
            row+=1
        for count in [0,1,2,3,4,5,6,7]:
            gempak_array[row,col] = line[8+count*9:17+count*9]
            col+=1

    file.close()
    
    return (gempak_array, -scale_factor)

In [ ]:
def error_stats(metpy_data, gempak_data):
    ignore = (gempak_data != -9999.0) * ~np.isnan(metpy_data)
    
    if isinstance(metpy_data.data, pint.Quantity):
        print(f'Is Xarray with Quantity? True')
        mdata = metpy_data.values[ignore]
    elif isinstance(metpy_data, pint.Quantity):
        print(f'Is Xarray with Quantity? False')
        print(f'Is Metpy Unit Array? True')
        mdata = metpy_data.m[ignore]
    else:
        print(f'Is Xarray with Quantity? False')
        print(f'Is Metpy Unit Array? False')
        print(f'I got back just a numpy array')
        try:
            mdata = metpy_data.values[ignore]
        except:
            mdata = metpy_data[ignore]
    gempak_data = gempak_data[ignore]
    
    print()
    print('Mean Comparison')
    print(f'  Mean Values (MetPy): {mdata.mean()}')
    print(f'  Mean Values (GEMPAK): {gempak_data.mean()}')
    print()
    print('Max Comparison')
    print(f'  Max Values (MetPy): {mdata.max()}')
    print(f'  Max Values (GEMPAK): {gempak_data.max()}')
    print()
    print('Min Comparison')
    print(f'  Min Values (MetPy): {mdata.min()}')
    print(f'  Min Values (GEMPAK): {gempak_data.min()}')

    print()
    print('Difference Array')
    diff = mdata - gempak_data
    print(diff)
    print()
    print('Various Statistical Analyses')
    print(f'  Average Absolute Difference: {np.nanmean(np.abs(diff))}')
    print(f'  RMS Error: {np.sqrt(np.nansum(diff**2))/len(diff.ravel())}')
    print(f'  Standard Deviation of Difference: {np.nanstd(diff)}')
    print(f'  Max Diff: {np.max(diff)}')
    print(f'  Min Diff: {np.min(diff)}')
    print(f'  Correlation: {np.corrcoef(mdata.ravel(), gempak_data.ravel())[0][1]}')
    print(f'  Relative Magnitude Difference: {np.nanmean(np.abs(diff))/np.nanmax(mdata)}')
    print()

In [ ]:
gfs_data = xr.open_dataset('gfs_test_data.nc').metpy.parse_cf()

In [ ]:
# Conversion from K to degC

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_tmpc.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_height_above_ground.sel(time=datetime(2018, 3, 8, 0))
data_test = data_var.metpy.convert_units('degC')

print('Conversion from K to degC \n')
error_stats(data_test, out_var)

In [ ]:
# Conversion from K to degf

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_tmpf.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_height_above_ground.sel(time=datetime(2018, 3, 8, 0))

data_test = data_var.metpy.convert_units('degF')

print('Conversion from K to degf \n')
error_stats(data_test, out_var)

In [ ]:
# Absolute Value testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_abs_function.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_height_above_ground.sel(time=datetime(2018, 3, 8, 0))

data_test = np.abs((data_var.metpy.convert_units('degC')))

print('Absolute Value \n')
error_stats(data_test, out_var)

In [ ]:
# Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))

data_test = mpcalc.wind_direction(data_var_u, data_var_v)

print('Wind Direction \n')
error_stats(data_test, out_var)

In [ ]:
# Cosine of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_cosine.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))

data_test = np.cos(np.deg2rad(mpcalc.wind_direction(data_var_u, data_var_v)))

print('Cosine of Wind Direction \n')
error_stats(data_test, out_var)

In [ ]:
# Sine of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_sine.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))

data_test = np.sin(np.deg2rad(mpcalc.wind_direction(data_var_u, data_var_v)))

print('Sine of Wind Direction \n')
error_stats(data_test, out_var)

In [ ]:
# Tangent of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_tangent.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))

data_test = np.tan(np.deg2rad(mpcalc.wind_direction(data_var_u, data_var_v)))

print('Tangent of Wind Direction \n')
error_stats(data_test, out_var)

In [ ]:
# ATAN
gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_arctangent.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000))

data_test = np.arctan(np.deg2rad(mpcalc.wind_direction(data_var_u, data_var_v)))

print('Arctangent of Wind Direction \n')
error_stats(data_test, out_var)

In [ ]:
# ATAN2 of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_atan2.out')

out_var = gempak_out_var * 10**scale

data_var_u = (gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)).metpy.quantify()
data_var_v = (gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)).metpy.quantify()

data_test = np.arctan2(data_var_u, data_var_v)

print('Arctangent2 of Wind Direction \n')
error_stats(data_test, out_var)

In [ ]:
# Arcsine of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_arcsine.out')

out_var = gempak_out_var * 10**scale

data_var_lat = gfs_data.lat.metpy.quantify()
data_var_lon = gfs_data.lon.metpy.quantify()

xx, yy = np.meshgrid(data_var_lon, data_var_lat)

data_test = np.arcsin((np.deg2rad(yy) * units.radian)/(np.pi / 2))

print('Arcsine of Wind Direction \n')
error_stats(data_test, out_var)

In [ ]:
# Arccosine of Wind Direction testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dirn_wind_arccosine.out')

out_var = gempak_out_var * 10**scale

data_var_lat = gfs_data.lat.values
data_var_lon = gfs_data.lon.values

xx, yy = np.meshgrid(data_var_lon, data_var_lat)

data_test = np.arccos((np.deg2rad(yy) * units.radian)/(np.pi / 2))

print('Arccosine of Wind Direction \n')
error_stats(data_test, out_var)

In [ ]:
# Exponential testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_exponential.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var ** 1.1

print('Exponential \n')
error_stats(data_test, out_var)

In [ ]:
# Integer Exponential testing

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_integer_exponential.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var ** np.rint(1.6)

print('Integer Exponential \n')
error_stats(data_test, out_var)

# NOTES: GEMPAK NINT(2.5) = 3; numpy.round(2.5) = 2.0 (as numpy rounds to nearest EVEN value); numpy.rint(2.5) = 2.0

In [ ]:
# Conversion integer of degf

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_integer_tmpf.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degF').astype('int')

print('Conversion integer of degf \n')
error_stats(data_test, out_var)

In [ ]:
# Natural Log of Geopotential Height

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_natural_log.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = np.log(data_var/units.meter)

print('Natural Log of Geopotential Height \n')
error_stats(data_test, out_var)

In [ ]:
# Log Base 10 of Geopotential Height

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_log_base10.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = np.log10(data_var/units.meter)

print('Log Base 10 of Geopotential Height \n')
error_stats(data_test, out_var)

In [ ]:
# Conversion round integer of degC

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_round_integer.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = np.rint(data_var.metpy.convert_units('degC'))

print('Conversion round integer of degC \n')
error_stats(data_test, out_var)

In [ ]:
# Square Root

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_square_root.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = np.sqrt(data_var)

print('Square Root \n')
error_stats(data_test, out_var)

In [ ]:
# Addition

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_add.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var + 5 * units.delta_degC

print('Addition \n')
error_stats(data_test, out_var)

In [ ]:
# Subtraction

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_subtract.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var - 5 * units.degC

print('Subtraction \n')
error_stats(data_test, out_var)

In [ ]:
# Multiplication

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_multiplication.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var * 2

print('Multiplication \n')
error_stats(data_test, out_var)

In [ ]:
# Quotient

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_quotient.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var / 2

print('Division \n')
error_stats(data_test, out_var)

In [ ]:
# Less Than

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_less_than.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degC')
data_mask = data_test < -10 * units.degC

print('Less Than -10 C \n')
#error_stats(data_test.where(data_mask).values.flatten(), out_var[data_mask])
print(data_test.where(data_mask).min().data)
print(out_var[data_mask.values].min())
print(data_test.where(data_mask).max().data)
print(out_var[data_mask.values].max())

In [ ]:
# Greater Than

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_greater_than.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degC')
data_mask = data_test > -10 * units.degC

print('Greater Than -10 C \n')
print(data_test.where(data_mask).min().data)
print(out_var[data_mask.values].min())
print(data_test.where(data_mask).max().data)
print(out_var[data_mask.values].max())

In [ ]:
# Less Than Equal To

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_less_than_equal.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degC')
data_mask = data_test <= -10.5 * units.degC

print('Less Than or Equal -10.5 C \n')
print(data_test.where(data_mask).min().data)
print(out_var[data_mask.values].min())
print(data_test.where(data_mask).max().data)
print(out_var[data_mask.values].max())

In [ ]:
# Greater Than

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_greater_than_equal.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degC')
data_mask = data_test >= -10.5 * units.degC

print('Greater Than or Equal -10.5 C \n')
print(data_test.where(data_mask).min().data)
print(out_var[data_mask.values].min())
print(data_test.where(data_mask).max().data)
print(out_var[data_mask.values].max())

In [ ]:
# Boolean Between

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_boolean_between.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_test = data_var.metpy.convert_units('degC')
data_mask = (data_test > -25 * units.degC) & (data_test < -10 * units.degC)

print('Boolean between -10 C and -25 C \n')
print(data_test.where(data_mask).min().data)
print(out_var[data_mask.values].min())
print(data_test.where(data_mask).max().data)
print(out_var[data_mask.values].max())

In [ ]:
# Advection

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_advection.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_degC = data_var.metpy.convert_units('degC')

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat
data_var_lon = gfs_data.lon

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

adv = mpcalc.advection(data_var_degC.data, [data_var_u.data, data_var_v.data], deltas=[dx, dy])

print('Advection \n')
error_stats(adv.to('degC/s')[2:-2,2:-2]*1e6, out_var[2:-2,2:-2]*1e6)

In [ ]:
# Average

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_average.out')

out_var = gempak_out_var * 10**scale

data_var_500 = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_850 = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

data_test = xr.concat([data_var_500, data_var_850], dim='level').mean(axis=0)

print('Average \n')
error_stats(data_test, out_var)

In [ ]:
# Corilois Parameter

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_coriolis.out')

out_var = gempak_out_var * 10**scale

data_var_lat = gfs_data.lat

data_test = mpcalc.coriolis_parameter(data_var_lat)

print('Coriolis Parameter \n')
error_stats(data_test[:180], out_var[:180, 0])


In [ ]:
# Absolute Vorticity

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_absolute_vorticity.out')

out_var = gempak_out_var * 10**scale

data_var_lat = gfs_data.lat

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

avor = mpcalc.absolute_vorticity(data_var_u, data_var_v, latitude=data_var_lat.values[:, None] * units.degree)

print('Absolute Vorticity \n')
error_stats(avor[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)

avor_corrected = avor + data_var_u / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat.values[:, None]))

print('\nAbsolute Vorticity with spherical corrections \n')
error_stats(avor_corrected[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)

In [ ]:
# Vorticity

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_vorticity.out')

out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat.values

vor = mpcalc.vorticity(data_var_u, data_var_v)

print('Vorticity \n')
error_stats(vor[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)

vor_corrected = vor + data_var_u / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None]))

print('\nVorticity with spherical corrections \n')
error_stats(vor_corrected[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)

In [ ]:
# Brunt Vaisala Frequency
gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_brunt_vaisala_freq.out')

out_var = gempak_out_var * 10**scale

data_var_p = gfs_data.isobaric2.metpy.sel(vertical=slice(55000, 45000))
data_var_hght = gfs_data.Geopotential_height_isobaric.metpy.sel(time=datetime(2018, 3, 8, 0), vertical=slice(55000, 45000))
data_var_tmpk = gfs_data.Temperature_isobaric.metpy.sel(time=datetime(2018, 3, 8, 0), vertical=slice(55000, 45000))

data_var_p_3D, _ = xr.broadcast(data_var_p, data_var_hght)

data_var_thta = mpcalc.potential_temperature(data_var_p, data_var_tmpk)

# layer = mpcalc.mixed_layer((data_var_p.data[::-1, None, None] * units.Pa).to('hPa'), data_var_thta.metpy.quantify(),
#                         height=data_var_hght.metpy.quantify(), bottom=850*units.hPa, depth=50*units.hPa)
# height_layer = data_var_hght.metpy.sel(vertical=500*units.hPa)-data_var_hght.metpy.sel(vertical=850*units.hPa)
data_test = mpcalc.brunt_vaisala_frequency_squared(data_var_hght.metpy.quantify().data, data_var_thta.data)
data_test1 = np.average(data_test, axis=0)
data_test2 = mpcalc.mixed_layer(data_var_p.values * units.Pa, data_test[:, 100, 100])

print('Brunt-Vaisala Frequency Squared \n')
#print(data_test2, out_var[100, 100])
#print(np.average(data_test[:, 100, 100]), out_var[100, 100])
error_stats(data_test1, out_var)

In [ ]:
# Cross Product

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_cross_product.out')

out_var = gempak_out_var * 10**scale


data_var_u_500 = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v_500 = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_u_850 = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()
data_var_v_850 = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

data_test = np.cross(xr.concat([data_var_u_850, data_var_v_850], dim='component'),
                     xr.concat([data_var_u_500, data_var_v_500], dim='component'), axis=0)

print('Cross Product of a Vector \n')
error_stats(data_test, out_var)

In [ ]:
# Density

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_density.out')

out_var = gempak_out_var * 10**scale

data_var_850 = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

data_test = mpcalc.density(850*units.hPa, data_var_850, mixing_ratio=0)

print('Density of Dry Air \n')
error_stats(data_test, out_var)

In [ ]:
# Partial Derivative in X

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_partial_x_derivative.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat
data_var_lon = gfs_data.lon

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

# Gives a unit_array, not a DataArray
data_test = mpcalc.first_derivative(data_var, delta=dx, axis=1)

print('Partial Derivative x-direction \n')
error_stats(data_test[3:-3,3:-3], out_var[3:-3,3:-3])

In [ ]:
# Partial Derivative in Y

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_partial_y_derivative.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat
data_var_lon = gfs_data.lon

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

data_test = mpcalc.first_derivative(data_var, delta=dy, axis=0)

print('Partial Derivative y-direction \n')
error_stats(data_test, out_var)

In [ ]:
# Deformation (shearing, stretching, total)

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_total_deformation.out')
out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat.values
data_var_lon = gfs_data.lon.values

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_shear_deformation.out')
out_var_shr = gempak_out_var * 10**scale

# Shearing Deformation
print()
shr_def = mpcalc.shearing_deformation(data_var_u, data_var_v)
print('Shearing Deformation \n')
error_stats(shr_def[3:-3,3:-3]*1e5, out_var_shr[3:-3,3:-3]*1e5)

print()
shr_def_corrected = (mpcalc.shearing_deformation(data_var_u, data_var_v)
                     + data_var_u / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))
print('Shearing Deformation with spherical corrections \n')
error_stats(shr_def_corrected[3:-3,3:-3]*1e5, out_var_shr[3:-3,3:-3]*1e5)

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_stretch_deformation.out')
out_var_str = gempak_out_var * 10**scale

# Streching Deformation
print()
str_def = mpcalc.stretching_deformation(data_var_u, data_var_v)
print('Stretching Deformation \n')
error_stats(str_def[3:-3,3:-3]*1e5, out_var_str[3:-3,3:-3]*1e5)

print()
str_def_corrected = (mpcalc.stretching_deformation(data_var_u, data_var_v)
                     - data_var_v / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))
print('Stretching Deformation with spherical corrections \n')
error_stats(str_def_corrected[3:-3,3:-3]*1e5, out_var_str[3:-3,3:-3]*1e5)

deformation = mpcalc.total_deformation(data_var_u, data_var_v)

print()
# Total Deformation
print('Total Deformation \n')
error_stats(deformation[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)



print()
deformation_corrected = np.sqrt(str_def_corrected**2 + shr_def_corrected**2)
print('Total Deformation with spherical corrections \n')
error_stats(deformation_corrected[3:-3,3:-3]*1e5, out_var[3:-3,3:-3]*1e5)

In [ ]:
# Divergence

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_divergence.out')

out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat.values
data_var_lon = gfs_data.lon.values

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

divergence = mpcalc.divergence(data_var_u, data_var_v)

print('Divergence \n')
error_stats(divergence[3:-3,3:-3]*1e4, out_var[3:-3,3:-3]*1e4)

divergence_corrected = (divergence - data_var_v / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))

print('\nDivergence with spherical corrections \n')
error_stats(divergence_corrected[3:-3,3:-3]*1e4, out_var[3:-3,3:-3]*1e4)

In [ ]:
# Dot Product

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_dot_product.out')

out_var = gempak_out_var * 10**scale


data_var_u_500 = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()
data_var_v_500 = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_u_850 = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()
data_var_v_850 = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

#data_test = data_var_u_850 * data_var_u_500 + data_var_v_850 * data_var_v_500
data_test2 = xr.dot(xr.concat([data_var_u_850, data_var_v_850], dim='component'),
                    xr.concat([data_var_u_500, data_var_v_500], dim='component'),
                    dims='component')

print('Dot Product of a Vector \n')
error_stats(data_test2, out_var)

In [ ]:
# Frotnogenesis

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_frontogenesis.out')

out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()
data_var_tmpk = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

thta = mpcalc.potential_temperature(850*units.hPa, data_var_tmpk)

data_var_lat = gfs_data.lat.values
data_var_lon = gfs_data.lon.values

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

fronto = mpcalc.frontogenesis(thta, data_var_u, data_var_v) * 1000 * 100 * 3600 * 3

# Frontogenesis undefined at a couple of points, ignore them for determining differences
#ignore = (out_var != -9999.0) * ~np.isnan(fronto)

print('Frontogenesis \n')
error_stats(fronto, out_var)

# Get gradients of potential temperature in both x and y
ddy_thta = mpcalc.first_derivative(thta, delta=dy, axis=-2)
ddx_thta = mpcalc.first_derivative(thta, delta=dx, axis=-1)

# Compute the magnitude of the potential temperature gradient
mag_thta = np.sqrt(ddx_thta**2 + ddy_thta**2)

# Get the shearing, stretching, and total deformation of the wind field
shrd = (mpcalc.shearing_deformation(data_var_u, data_var_v) 
        + data_var_u / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))
strd = (mpcalc.stretching_deformation(data_var_u, data_var_v)
        - data_var_v / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))
tdef = np.sqrt(shrd**2 + strd**2)

# Get the divergence of the wind field
div = (mpcalc.divergence(data_var_u, data_var_v)
       - data_var_v / mpconst.earth_avg_radius * np.tan(np.deg2rad(data_var_lat[:, None])))

# Compute the angle (beta) between the wind field and the gradient of potential temperature
psi = 0.5 * np.arctan2(shrd, strd)
beta = np.arcsin((-ddx_thta * np.cos(psi) - ddy_thta * np.sin(psi)) / mag_thta)

fronto_corrected = 0.5 * mag_thta * (tdef * np.cos(2 * beta) - div) * 1000 * 100 * 3600 * 3

print()
print('Frontogenesis with spherical corrections \n')
error_stats(fronto_corrected, out_var)

In [ ]:
# To Knots

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_to_knots.out')

out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

wspd = mpcalc.wind_speed(data_var_u, data_var_v)

wspd_kts = wspd.metpy.convert_units('kt')
wspd_kts = wspd * 1.9425 # Actual GEMPAK conversion

print('Knots Unit Conversion \n')
error_stats(wspd_kts, out_var)

In [ ]:
# Laplacian

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_laplacian.out')

out_var = gempak_out_var * 10**scale

data_var_hght = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var_lat = gfs_data.lat
data_var_lon = gfs_data.lon

dx, dy = mpcalc.lat_lon_grid_deltas(data_var_lon, data_var_lat)

data_test = mpcalc.laplacian(data_var_hght.metpy.unit_array, deltas=(dy, dx))

dzdy, dzdx = mpcalc.gradient(data_var_hght, deltas=(dy, dx))
data_test2 = mpcalc.divergence(dzdx, dzdy, dx, dy)

print('Laplacian with MetPy Function \n')
error_stats(data_test[3:-3,3:-3]*10**7, out_var[3:-3,3:-3]*10**7)

print('\nLaplacian calculated the GEMPAK way \n')
error_stats(data_test2[3:-3,3:-3]*10**7, out_var[3:-3,3:-3]*10**7)

In [ ]:
# Magnitude

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_magnitude.out')

out_var = gempak_out_var * 10**scale

data_var_u = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
data_var_v = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

data_test = mpcalc.wind_speed(data_var_u, data_var_v)

print('Magnitude \n')
error_stats(data_test, out_var)

In [ ]:
# Mixing Ratio

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_mixing_ratio.out')

out_var = gempak_out_var * 10**scale
# Set GEMPAK missing values to 0 RH
out_var[out_var == -9999] = 0.0

data_var_rh = gfs_data['Relative_humidity_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000)
data_var_tmpk = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=85000)

mixr = mpcalc.mixing_ratio_from_relative_humidity(850*units.hPa, data_var_tmpk, data_var_rh)

print('Mixing Ratio \n')
error_stats(mixr, out_var/1000)

In [ ]:
# SM5S

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_sm5s_filter.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

smooth_data_var = mpcalc.smooth_n_point(data_var)

print('5-point Smoothing \n')
error_stats(smooth_data_var[1:-1, 1:-1], out_var[1:-1, 1:-1])

In [ ]:
# SM9S

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_sm9s_filter.out')

out_var = gempak_out_var * 10**scale

data_var = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

smooth_data_var = mpcalc.smooth_n_point(data_var, 9)

print('9-point Smoothing \n')
error_stats(smooth_data_var[1:-1, 1:-1], out_var[1:-1, 1:-1])

In [ ]:
# PVOR
gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_pvor_500.out')

out_var = gempak_out_var * 10**scale

lats = gfs_data.lat.values * units.degree
lons = gfs_data.lon.values * units.degree

p = gfs_data.Geopotential_height_isobaric.isobaric2
T = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0))

uwnd = gfs_data['u-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0))
vwnd = gfs_data['v-component_of_wind_isobaric'].sel(time=datetime(2018, 3, 8, 0))

potemp = mpcalc.potential_temperature(p, T)

pvor = mpcalc.potential_vorticity_baroclinic(potemp, p, uwnd, vwnd, latitude=lats[None, :, None])


# Begin Spherical Correction Calculation


dx, dy = mpcalc.lat_lon_grid_deltas(lons, lats)

avor = (mpcalc.absolute_vorticity(uwnd, vwnd, latitude=lats[None, :, None]) 
        + uwnd.metpy.quantify() / mpconst.earth_avg_radius * np.tan(np.deg2rad(lats[None, :, None])))
dthtadp = mpcalc.first_derivative(potemp, x=p.values[:, None, None] * units.Pa, axis=-3)

if ((np.shape(potemp)[-2] == 1)
   and (np.shape(potemp)[-1] == 1)):
    dthtady = 0 * units.K / units.m  # axis=-2 only has one dimension
    dthtadx = 0 * units.K / units.m  # axis=-1 only has one dimension
else:
    dthtady = mpcalc.first_derivative(potemp, delta=dy[None, :, :], axis=-2)
    dthtadx = mpcalc.first_derivative(potemp, delta=dx[None, :, :], axis=-1)
dudp = mpcalc.first_derivative(uwnd, x=p.values[:, None, None] * units.Pa, axis=-3)
dvdp = mpcalc.first_derivative(vwnd, x=p.values[:, None, None] * units.Pa, axis=-3)

pvor_corrected = (-mpconst.g * (dudp * dthtady - dvdp * dthtadx
                       + avor * dthtadp)).data.to_base_units()

data_var = pvor[-2]

print('Baroclinic Potential Vorticity Calculation \n')
error_stats(data_var[3:-3, 3:-3]*10**6, out_var[3:-3, 3:-3]*10**6)

print()
print('Baroclinic Potential Vorticity with spherical corrections \n')
error_stats(pvor_corrected[-2, 3:-3, 3:-3]*10**6, out_var[3:-3, 3:-3]*10**6)

In [ ]:
# Potential Temperature

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_potential_temperature.out')

out_var = gempak_out_var * 10**scale

T = gfs_data.Temperature_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
potemp = mpcalc.potential_temperature(500 * units.hPa, T)

print('Potential Temperature MetPy Calcualation \n')
error_stats(potemp, out_var)

print()
print('Potential Temperature using GEMPAK conversion (R/Cp = 2/7) \n')
potemp2 = T * (1000./500.)**(2/7)
error_stats(potemp2, out_var)

In [ ]:
# Relative Humidity

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_relhum.out')

out_var = gempak_out_var * 10**scale

T = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
rh = gfs_data['Relative_humidity_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

dwpc = mpcalc.dewpoint_from_relative_humidity(T, rh)

data_var = mpcalc.relative_humidity_from_dewpoint(T, dwpc) * 100

print('Relative Humidity \n')
error_stats(data_var, out_var)

In [ ]:
# Max over grid

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_sgmx.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

# np.nanmax takes DataArray to numpy float
data_var = np.round(np.max(hght), decimals=2)

print('Max over Grid \n')
error_stats(data_var, out_var[0, 0])

In [ ]:
# Min over grid

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_sgmn.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var = np.round(np.min(hght), decimals=2)

print('Min over grid \n')
error_stats(data_var, out_var[0, 0])

In [ ]:
# Saturated Potential Temperature

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_thes.out')

out_var = gempak_out_var * 10**scale

T = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

data_var = mpcalc.saturation_equivalent_potential_temperature(500 * units.hPa, T)

print('Saturated Potential Temperature \n')
error_stats(data_var, out_var)

In [ ]:
# Equivalent Potential Temperature

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_thte.out')

out_var = gempak_out_var * 10**scale

T = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)
rh = gfs_data['Relative_humidity_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000)

dwpc = mpcalc.dewpoint_from_relative_humidity(T, rh)

data_var = mpcalc.equivalent_potential_temperature(500 * units.hPa, T, dwpc)

print('Equivalent Potential Temperature \n')
error_stats(data_var, out_var)


In [ ]:
# Wet Bulb Temperature - output units?

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_thwc.out')

out_var = gempak_out_var * 10**scale

T = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=100000)
rh = gfs_data['Relative_humidity_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=100000)

dwpc = mpcalc.dewpoint_from_relative_humidity(T.metpy.unit_array, rh.metpy.unit_array)

data_var = mpcalc.wet_bulb_temperature(1000 * units.hPa, T[25, 101], dwpc[25, 101])

print('Wet Bulb Temperature \n')
error_stats(data_var.metpy.convert_units('degC'), out_var[25, 101])
#print(np.nanmean(data_var))

In [ ]:
# Moist Lapse Rate

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_tmst.out')

out_var = gempak_out_var * 10**scale

T = gfs_data['Temperature_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=100000)
rh = gfs_data['Relative_humidity_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=100000)

dwpc = mpcalc.dewpoint_from_relative_humidity(T, rh)

thte = mpcalc.equivalent_potential_temperature(500 * units.hPa, T, dwpc)

data_var = mpcalc.moist_lapse([500] * units.hPa, T[100, 100], 1000 * units.hPa)

error_stats(data_var, out_var[100, 100])
print(data_var)
print(out_var[200, 200])

In [ ]:
# Average over row

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_xav.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var = np.round(np.mean(hght, axis=1), decimals=2)

print('Average over row \n')
error_stats(data_var, out_var[:, 0])

In [ ]:
# Average over column

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_yav.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data['Geopotential_height_isobaric'].sel(time=datetime(2018, 3, 8, 0), isobaric2=50000).metpy.quantify()

data_var = np.round(np.mean(hght, axis=0), decimals=2)

print('Average over column \n')
error_stats(data_var, out_var[0, :])

In [ ]:
# Sum over column

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_ysum.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

data_var = np.round(np.sum(hght, axis=0), decimals=2)

print('Sum over column')
error_stats(data_var, out_var[0, :])

In [ ]:
# Sum over row

gempak_out_var, scale = read_gdlist('gempak_out_files/gempak_xsum.out')

out_var = gempak_out_var * 10**scale

hght = gfs_data.Geopotential_height_isobaric.sel(time=datetime(2018, 3, 8, 0), isobaric2=85000).metpy.quantify()

data_var = np.round(np.sum(hght, axis=1), decimals=2)

print('Sum over row \n')
error_stats(data_var, out_var[:, 0])